In [3]:

!pip install geopy
!pip install folium
!pip install geocoder
print("Installed!")

Installed!


<h3>Use geopy library to get the latitude and longitude values of Toronto City</h3>

In [2]:
import folium
import requests 
import json 

import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import numpy as np
import geocoder
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)[0]

In [5]:
# clean dataframe 
df_list = df_list[df_list.Borough!='Not assigned']
df_list = df_list[df_list.Borough!=0]
df_list.reset_index(drop = True, inplace = True)

i = 0
for i in range(0,df_list.shape[0]):
    if df_list.iloc[i][2] == 'Not assigned':
        df_list.iloc[i][2] = df_list.iloc[i][1]
        i = i+1

In [6]:
df = df_list.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(', '.join).reset_index()


In [7]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords

In [8]:

# Retrieving Postal Code Co-ordinates
postal_codes = df['Postal Code']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [9]:
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [10]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [31]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


In [35]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [37]:
scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470
3,M1G,Scarborough,Woburn,43.76812,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892


In [38]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


In [41]:

# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Borough'], scarborough_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

<b>Define Foursquare Credentials and Version

In [42]:

CLIENT_ID = 'EM1DPOY2TSWUIQJJQ42LCCT3XY4SLA0KCOWZU0DHQTZR3DOT' # your Foursquare ID
CLIENT_SECRET = 'EBUWTCYITSH2GLCPIKEPC1WY1GT4GY4Y5DDYRZP5ACLEDROY' # your Foursquare Secret
VERSION = '20201229' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EM1DPOY2TSWUIQJJQ42LCCT3XY4SLA0KCOWZU0DHQTZR3DOT
CLIENT_SECRET:EBUWTCYITSH2GLCPIKEPC1WY1GT4GY4Y5DDYRZP5ACLEDROY


<b>Let's explore the first neighborhood in our dataframe

In [44]:

scarborough_data.loc[0, 'Neighbourhood']

'Malvern, Rouge'

<b>Get the neighborhood's latitude and longitude values

In [46]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.811390000000074, -79.19661999999994.


<b>Now, let's get the top 100 venues that are in Malvern , Rouge within a radius of 500 meters¶


In [47]:
LIMIT = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5feaeb3f87013231d67df8a4'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.815890004500076,
    'lng': -79.19039569057789},
   'sw': {'lat': 43.80688999550007, 'lng': -79.20284430942199}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label':

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<b>
Clean the json and structure it into a pandas dataframe

In [50]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-50-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [51]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


<b>Function to repeat the same process to all the neighborhoods in Scarborough¶


In [90]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [91]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [92]:

print(scarborough_venues.shape)
scarborough_venues.head()

(87, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.81139,-79.19662,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.76575,-79.17470,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.76575,-79.17470,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center


<b>No. of venues  returned for each neighborhood¶


In [93]:
scarborough_venues.groupby('Neighbourhood').count()


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,13,13,13,13,13,13
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,1,1,1,1,1,1
"Clarks Corners, Tam O'Shanter, Sullivan",13,13,13,13,13,13
"Cliffside, Cliffcrest, Scarborough Village West",10,10,10,10,10,10
"Dorset Park, Wexford Heights, Scarborough Town Centre",3,3,3,3,3,3
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",3,3,3,3,3,3
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


<b>Unique categories curated from all the returned venues

In [94]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))


There are 49 uniques categories.


<h2>Analyze Each Neighborhood¶
</h2>

In [95]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Bubble Tea Shop,Bus Line,Bus Station,Business Service,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,General Entertainment,Gift Shop,Grocery Store,Gym / Fitness Center,Hardware Store,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Korean BBQ Restaurant,Light Rail Station,Liquor Store,Metro Station,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Trail,Video Game Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [96]:
scarborough_onehot.shape

(87, 50)

<b>Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶


In [97]:
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped

,Neighbourhood,Auto Garage,Badminton Court,Bakery,Bank,Bar,Bubble Tea Shop,Bus Line,Bus Station,Business Service,Café,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,General Entertainment,Gift Shop,Grocery Store,Gym / Fitness Center,Hardware Store,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Korean BBQ Restaurant,Light Rail Station,Liquor Store,Metro Station,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Trail,Video Game Store,Vietnamese Restaurant
0,Agincourt,0.0,0.076923,0.076923,0.000000,0.0,0.076923,0.000000,0.000000,0.00,0.00,0.000000,0.153846,0.000000,0.00,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.00,0.000000,0.076923,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.076923,0.076923,0.000000,0.076923,0.076923,0.0,0.000000,0.076923
1,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.25,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,Cedarbrae,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.076923,0.000000,0.076923,0.00,0.000000,0.076923,0.076923,0.000000,0.076923,0.000000,0.153846,0.00,0.000000,0.076923,0.000000,0.0,0.076923,0.0,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.076923,0.00,0.00,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.076923,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.100000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.1,0.000000,0.2,0.00,0.000000,0.00,0.000000,0.1,0.000000,0.000000,0.000000,0.100000,0.100000,0.00,0.10,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.0,0.000000,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,"Golden Mile, Clairlea, Oakridge",0.0,0.000000,0.222222,0.000000,0.0,0.000000,0.111111,0.111111,0.00,0.00,0.000000,0.000000,0.111111,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.222222,0.00,0.000000,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.333333,0.0,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00000

In [98]:
scarborough_grouped.shape

(16, 50)

<b>Print each neighborhood along with the top 5 most common venues

In [99]:
num_top_venues = 5

for hood in scarborough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                   venue  freq
0     Chinese Restaurant  0.15
1  Vietnamese Restaurant  0.08
2            Supermarket  0.08
3          Grocery Store  0.08
4          Shopping Mall  0.08


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4            Pizza Place  0.00


----Cedarbrae----
               venue  freq
0              Trail   1.0
1        Auto Garage   0.0
2           Pharmacy   0.0
3     Ice Cream Shop   0.0
4  Indian Restaurant   0.0


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0  Fast Food Restaurant  0.15
1        Discount Store  0.08
2      Video Game Store  0.08
3                  Bank  0.08
4           Pizza Place  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
            venue  freq
0  Ice Cream Shop   0.2
1     Pizza Place   0.1
2  Sandwich Place   0.1
3    L

<b>Put this into a pandas dataframe¶


In [100]:
#function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [102]:
#create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Skating Rink,Department Store,Discount Store,Bubble Tea Shop,Shopping Mall,Vietnamese Restaurant,Bakery,Supermarket,Sushi Restaurant
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,Cedarbrae,Trail,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega,Convenience Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Fast Food Restaurant,Pizza Place,Grocery Store,Video Game Store,Hobby Shop,Discount Store,Deli / Bodega,Convenience Store,Coffee Shop,Cantonese Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",Ice Cream Shop,Liquor Store,Discount Store,Hardware Store,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Coffee Shop,Vietnamese Restaurant


<h2>Cluster Neighborhoods
</h2>

In [103]:
#Run k-means to cluster the neighborhood into 5 clusters

# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 3, 1, 1, 1, 1, 0, 1, 4])

In [104]:

#create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
neighbourhoods_venues_sorted 

# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarborough_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,4.0,Fast Food Restaurant,Vietnamese Restaurant,Grocery Store,General Entertainment,Electronics Store,Discount Store,Department Store,Deli / Bodega,Convenience Store,Construction & Landscaping
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78574,-79.15875,0.0,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17470,0.0,Gym / Fitness Center,Park,Construction & Landscaping,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega,Convenience Store
3,M1G,Scarborough,Woburn,43.76812,-79.21761,1.0,Park,Korean BBQ Restaurant,Business Service,Coffee Shop,Vietnamese Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega
4,M1H,Scarborough,Cedarbrae,43.76944,-79.23892,3.0,Trail,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega,Convenience Store


In [11]:

#visualize the resulting clusters
import numpy as np
import matplotlib.cm as cm
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'latitude' is not defined

<b> Examine Cluster

<b>Cluster 1

In [113]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Bar,Construction & Landscaping,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega
2,Scarborough,0.0,Gym / Fitness Center,Park,Construction & Landscaping,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega,Convenience Store


<b>Cluster 2

In [114]:

scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,1.0,Park,Korean BBQ Restaurant,Business Service,Coffee Shop,Vietnamese Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega
5,Scarborough,1.0,Park,Restaurant,Indian Restaurant,Grocery Store,Bank,Bar,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,Scarborough,1.0,Park,Department Store,Playground,Coffee Shop,Vietnamese Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Deli / Bodega,Convenience Store
7,Scarborough,1.0,Bakery,Intersection,Soccer Field,Bus Line,Bus Station,Coffee Shop,Metro Station,Vietnamese Restaurant,College Stadium,Fast Food Restaurant
8,Scarborough,1.0,Ice Cream Shop,Liquor Store,Discount Store,Hardware Store,Sandwich Place,Restaurant,Pizza Place,Pharmacy,Coffee Shop,Vietnamese Restaurant
9,Scarborough,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
10,Scarborough,1.0,Gift Shop,Bakery,Light Rail Station,Coffee Shop,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega
11,Scarborough,1.0,Auto Garage,Convenience Store,Grocery Store,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega,Construction & Landscaping
12,Scarborough,1.0,Chinese Restaurant,Skating Rink,Department Store,Discount Store,Bubble Tea Shop,Shopping Mall,Vietnamese Restaurant,Bakery,Supermarket,Sushi Restaurant
13,Scarborough,1.0,Fast Food Restaurant,Pizza Place,Grocery Store,Video Game Store,Hobby Shop,Discount Store,Deli / Bodega,Convenience Store,Coffee Shop,Cantonese Restaurant


<b>Cluster 3 

In [115]:

scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,2.0,Pharmacy,Intersection,Chinese Restaurant,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega,Convenience Store,Construction & Landscaping


<b>Cluster 4

In [116]:

scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,3.0,Trail,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Deli / Bodega,Convenience Store


<b>Cluster 5

In [117]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Vietnamese Restaurant,Grocery Store,General Entertainment,Electronics Store,Discount Store,Department Store,Deli / Bodega,Convenience Store,Construction & Landscaping
